# Miniproject1 - K Nearest Neighbors Approach

In [1]:
import numpy as np 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

In [2]:
cd E:\Caltech_Winter\CS 155\mini project1\data

E:\Caltech_Winter\CS 155\mini project1\data


## Importing Data

In [3]:
from numpy import genfromtxt
import numpy as np
train_file = 'training_data.txt'
test_file = 'test_data.txt'
train_data = genfromtxt(train_file, delimiter=' ')
test_data = genfromtxt(test_file, delimiter=' ')
train_data = train_data[1:]

np.random.seed(123456)
ind = np.random.permutation(train_data.shape[0])
new_train_data = train_data[ind].astype(int)

X_train = new_train_data[:,1:]
y_train = new_train_data[:,0]

# XX = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
pca=PCA(copy=True, iterated_power='auto', n_components=500, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)
pca.fit(X_train)

X_pca=pca.transform(X_train)
print(X_pca.shape)

(20000, 500)


## Cross Validation Function

In [15]:
def CrossValidation(X_train, y_train, n_neighbors, weights, fold):

    kf = KFold(n_splits=fold)
    max_acc = 0
    i = 1
    sum_acc = 0
    
    print('Parameters: [N_neighbors] %d, [Weights] %s, [Fold] %d' 
          % (n_neighbors, weights, fold))
    print('Start Training...')
    
    for train_index, test_index in kf.split(X_train):
        if i == 2:
            break
        print('Train on Fold %1d' % i)
        i = i + 1
        
        KNN = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
        KNN.fit(X_train[train_index], y_train[train_index])
        
        train_acc = KNN.score(X_train[train_index], y_train[train_index])
        test_acc = KNN.score(X_train[test_index], y_train[test_index])
        
        print('Training accuracy:', train_acc)
        print('Test accuracy:', test_acc)
        
        sum_acc += test_acc
        if test_acc > max_acc:
            max_acc = test_acc
            
    print('Best accuracy:', max_acc, 'Avg accuracy:', sum_acc / fold)

## Testing Parameters

In [22]:
# train_data1 = train_data[1:100]
fold = 5
n_neighbors = [5, 10, 20, 50]
weights = 'uniform'
for n in n_neighbors:
    CrossValidation(X_pca, y_train, n, weights, fold)

Parameters: [N_neighbors] 5, [Weights] uniform, [Fold] 5
Start Training...
Train on Fold 1
Training accuracy: 0.782375
Test accuracy: 0.6515
Best accuracy: 0.6515 Avg accuracy: 0.1303
Parameters: [N_neighbors] 10, [Weights] uniform, [Fold] 5
Start Training...
Train on Fold 1
Training accuracy: 0.7514375
Test accuracy: 0.6765
Best accuracy: 0.6765 Avg accuracy: 0.1353
Parameters: [N_neighbors] 20, [Weights] uniform, [Fold] 5
Start Training...
Train on Fold 1
Training accuracy: 0.745625
Test accuracy: 0.69475
Best accuracy: 0.69475 Avg accuracy: 0.13895
Parameters: [N_neighbors] 50, [Weights] uniform, [Fold] 5
Start Training...
Train on Fold 1
Training accuracy: 0.738375
Test accuracy: 0.714
Best accuracy: 0.714 Avg accuracy: 0.14279999999999998


## Without PCA

In [21]:
fold = 5
n_neighbors = [5, 10, 20, 50]
weights = 'uniform'
for n in n_neighbors:
    CrossValidation(X_train, y_train, n, weights, fold)

Parameters: [N_neighbors] 5, [Weights] uniform, [Fold] 5
Start Training...
Train on Fold 1
Training accuracy: 0.7409375
Test accuracy: 0.64275
Best accuracy: 0.64275 Avg accuracy: 0.12855
Parameters: [N_neighbors] 10, [Weights] uniform, [Fold] 5
Start Training...
Train on Fold 1
Training accuracy: 0.755125
Test accuracy: 0.66975
Best accuracy: 0.66975 Avg accuracy: 0.13394999999999999
Parameters: [N_neighbors] 20, [Weights] uniform, [Fold] 5
Start Training...
Train on Fold 1
Training accuracy: 0.7400625
Test accuracy: 0.68725
Best accuracy: 0.68725 Avg accuracy: 0.13745000000000002
Parameters: [N_neighbors] 50, [Weights] uniform, [Fold] 5
Start Training...
Train on Fold 1
Training accuracy: 0.7089375
Test accuracy: 0.6805
Best accuracy: 0.6805 Avg accuracy: 0.1361


## Try Bagging (without PCA)

In [26]:
y_stack = np.zeros(len(y_train))
fold = 5
n_neighbors = 20
weights = 'uniform'

kf = KFold(n_splits=fold)

for train_index, test_index in kf.split(X_train):
    KNN = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
    KNN.fit(X_train[train_index], y_train[train_index])
    y_stack[test_index] = KNN.predict(X_train[test_index])

In [27]:
y_stack_int = [1 if i > 0.5 else 0 for i in y_stack]

with open("KNN.txt", 'a') as text_file:
    text_file.write("Id,Prediction\n")
    for i in range(len(y_stack_int)):
        text_file.write("{},{}\n".format(i+1, y_stack_int[i]))

In [28]:
X_test = test_data[1:]
KNN = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
KNN.fit(X_train, y_train)
y_test = KNN.predict(X_test)

In [29]:
y_test_int = [1 if i > 0.5 else 0 for i in y_test]

with open("KNNTest.txt", 'a') as text_file:
    text_file.write("Id,Prediction\n")
    for i in range(len(y_test_int)):
        text_file.write("{},{}\n".format(i+1, y_test_int[i]))

## Try Bagging (with PCA)

In [4]:
y_stack = np.zeros(len(y_train))
fold = 5
n_neighbors = 50
weights = 'uniform'

kf = KFold(n_splits=fold)

for train_index, test_index in kf.split(X_train):
    KNN = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
    KNN.fit(X_pca[train_index], y_train[train_index])
    y_stack[test_index] = KNN.predict(X_pca[test_index])

In [5]:
y_stack_int = [1 if i > 0.5 else 0 for i in y_stack]

with open("KNNPCA.txt", 'a') as text_file:
    text_file.write("Id,Prediction\n")
    for i in range(len(y_stack_int)):
        text_file.write("{},{}\n".format(i+1, y_stack_int[i]))

In [6]:
X_test = test_data[1:]
X_test_pca = pca.transform(X_test)
KNN = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
KNN.fit(X_pca, y_train)
y_test = KNN.predict(X_test_pca)

In [7]:
y_test_int = [1 if i > 0.5 else 0 for i in y_test]

with open("KNNPCATest.txt", 'a') as text_file:
    text_file.write("Id,Prediction\n")
    for i in range(len(y_test_int)):
        text_file.write("{},{}\n".format(i+1, y_test_int[i]))